In [1]:
import numpy as np 
import scipy.io as sio
import matplotlib.pyplot as plt
import xarray as xr
import os
import cftime
import datetime as dt


from tqdm import tqdm, notebook

In [3]:
# reading in LEOC96 data
in_folder = '/mnt/drive3/gfdl/6HRLY/SURF/'

var = 'ua'

var_map = {'ua': 'uas', 'va': 'vas', 'wap': 'wap'}
var_units = {'ua': 'm s-1', 'va': 'm s-1', 'wap': 'Pa/s'}
var_scale = {'ua': 1., 'va': 1., 'wap': 1.}


out_var_map = {'ua': 'ua',  'va': 'va', 'wap': 'wap'}
out_var_units = {'ua': 'm s-1',  'va': 'm s-1', 'wap': 'Pa/s'}
out_var_scale = {'ua': 1.,  'va': 1., 'wap': 1.} 
out_var_ln = {'ua': 'Eastward Wind',  'va': 'Westward Wind', 'wap': 'Vertical Velocity'} 
out_var_sn = {'ua': 'eastward_wind',  'va': 'westward_wind', 'wap': 'lagrangian_tendency_of_air_pressure'} 

in_file = f'/mnt/drive3/gfdl/6HRLY/SURF/atmos.2008010100-2012123123.{var_map[var]}.nc'

print(in_file)
print(os.path.exists(in_file))

/mnt/drive3/gfdl/6HRLY/SURF/atmos.2008010100-2012123123.uas.nc
True


In [ ]:
%%time
in_ds = xr.open_dataset(in_file)
in_lat = in_ds.variables['lat'].values
in_lon = in_ds.variables['lon'].values

In [ ]:
year_range = [2008, 2012]

time_start_year = 1975

# must include the end year, so have to add one 
num_years = year_range[1] - year_range[0] + 1

# creating the output arrays
out_time_bnds = np.zeros((1460*num_years, 2))

# time variable for all the years
in_time = np.arange(0, num_years*365, .25) + (year_range[0] - time_start_year)*365 
print(len(in_time))

In [ ]:
out_time = in_time

lon = np.copy(in_lon)
dlon = lon[1] - lon[0]
out_lon = np.round(lon - dlon/2., decimals=2)

lat = np.copy(in_lat)
dlat = lat[1] - lat[0]
out_lat = np.round(lat - dlat/2., decimals=2)

# getting the output time bounds
out_time_bnds[:, 1] = out_time
out_time_bnds[0, 0] = out_time[0]
out_time_bnds[1:, 0] = out_time[:-1]

print(out_time_bnds.shape, out_lat.shape, out_lon.shape)

In [22]:
# time_arr = [cftime.DatetimeNoLeap(time_start_year, 1, 1) + dt.timedelta(days=i) for i in tqdm.notebook.tqdm(out_time, total=len(out_time))]
out_date = np.empty(out_time.shape, dtype=int)
for i, i_time in notebook.tqdm(enumerate(out_time), total=len(out_time)): 
    tmp = cftime.DatetimeNoLeap(time_start_year, 1, 1) + dt.timedelta(days=i_time)
    out_date[i] = int('%04d%02d%02d'%(tmp.year, tmp.month, tmp.day))

In [28]:
print(out_date)
print(in_ds.pfull.shape)
print(in_ds.phalf.shape)
print(in_ds[var_map[var]].shape)

[20080101 20080101 20080101 ... 20121231 20121231 20121231]
(32,)
(33,)
(7308, 32, 180, 288)


In [54]:
ds = in_ds.copy()

# print(in_ds)
ds = ds.isel(time=range(len(out_time)))
ds['time'] = out_time
ds['lat'] = out_lat
ds['lon'] = out_lon
ds['nbnd'] = [1, 2]
ds['date'] = out_date
ds['time_bnds'] = (['time', 'nbnd'], out_time_bnds)


ds = ds.rename({var_map[var]: out_var_map[var], 'pfull': 'lev'})

ds = ds.drop(['time_bounds', 'lat_bnds', 'lon_bnds', 'phalf'])

# lon attribs
ds.lon.attrs['standard_name'] = 'longitude'
ds.lon.attrs['long_name'] = 'longitude'
ds.lon.attrs['units'] = 'degrees_east'

# lat attribs
ds.lat.attrs['standard_name'] = 'latitude'
ds.lat.attrs['long_name'] = 'latitude'
ds.lat.attrs['units'] = 'degrees_north'

# time attribs
ds.time.attrs['standard_name'] = 'time'
ds.time.attrs['long_name'] = 'time'
ds.time.attrs['units'] = 'days since 1975-01-01 00:00:00'
ds.time.attrs['calendar'] = 'noleap'
ds.time.attrs['bounds'] = 'time_bnds'

# time_bnds attribs
ds.time_bnds.attrs['long_name'] = 'time interval endpoints'

# date attribs
ds.date.attrs['long_name'] = 'current date (YYYYMMDD)'

# slp attribs
ds[out_var_map[var]].attrs['long_name'] = out_var_ln[var]
ds[out_var_map[var]].attrs['standard_name'] = out_var_sn[var]
ds[out_var_map[var]].attrs['units'] = out_var_units[var]

print(ds.wap)

print('---------------------------')
print(ds)
# print(ds[out_var_map[var]])

<xarray.DataArray 'wap' (time: 7300, lev: 32, lat: 180, lon: 288)>
[12109824000 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 -90.0 -89.0 -88.0 -87.0 -86.0 ... 86.0 87.0 88.0 89.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * lev      (lev) float64 2.164 5.845 10.75 17.11 ... 963.7 976.7 987.4 996.1
  * time     (time) float64 1.204e+04 1.205e+04 ... 1.387e+04 1.387e+04
Attributes:
    long_name:      Vertical Velocity
    units:          Pa/s
    cell_methods:   time: point
    interp_method:  conserve_order2
    standard_name:  lagrangian_tendency_of_air_pressure
---------------------------
<xarray.Dataset>
Dimensions:    (date: 7300, lat: 180, lev: 32, lon: 288, nbnd: 2, time: 7300)
Coordinates:
  * lat        (lat) float64 -90.0 -89.0 -88.0 -87.0 ... 86.0 87.0 88.0 89.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * lev        (lev) float64 2.164 5.845 10.75 17.11 ... 963.7 976.7 987.4 996.1

In [ ]:
out_file = f'/localdrive/drive10/jj/mdtf/inputdata/model/GFDL.EXP1.2PM.001/6hr/GFDL.EXP1.2PM.001.{out_var_map[var]}.6hr.nc'
print(out_file)
ds.to_netcdf(out_file)

/localdrive/drive10/jj/mdtf/inputdata/model/GFDL.EXP1.2PM.001/6hr/GFDL.EXP1.2PM.001.wap.6hr.nc
